In [1]:
import keras
import wandb
import os
from wandb.keras import WandbCallback
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
import DatasetParser
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.
wandb: WARNING Keras version 2.3.1 is not fully supported. Required keras >= 2.4.0


In [2]:
(x_train, y_train_raw), (x_test, y_test_raw), class_names = DatasetParser.load_data("Inception")

299
Entrenamiento: 3348
Test: 1440


In [3]:
# One hot encode ouput
y_train = keras.utils.to_categorical(y_train_raw)
y_test = keras.utils.to_categorical(y_test_raw)

In [4]:
Inception_model = keras.applications.InceptionV3(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=True)

In [5]:
from keras.preprocessing import image
import numpy as np
img = image.load_img('elephant.jpg', target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = Inception_model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n04286575', 'spotlight', 0.54848224), ('n03950228', 'pitcher', 0.43501875), ('n01740131', 'night_snake', 0.016497778)]


In [6]:
# We should preprocess the images the same way resnet images were preprocessed
x_train_preprocessed = preprocess_input(x_train)
x_test_preprocessed = preprocess_input(x_test)

In [7]:
# Build a new model that is ResNet50 minus the very last layer
last_layer = Inception_model.get_layer("avg_pool")

Inception_layers = keras.Model(inputs=Inception_model.inputs, outputs=last_layer.output)
Inception_layers.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [8]:
# We use our resnet to "predict" but because we have removed the top layer, 
# this outputs the activations of the second to last layer on our dataset

x_train_features = Inception_layers.predict(x_train_preprocessed)

In [9]:
x_test_features = Inception_layers.predict(x_test_preprocessed)

In [10]:
# We can directly stich the models together

Fine_Tuning_model=Sequential()
Fine_Tuning_model.add(Inception_layers)
Fine_Tuning_model.add(Dense(36, activation="softmax"))

Fine_Tuning_model.layers[0].trainable=False

Fine_Tuning_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Fine_Tuning_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 2048)              21802784  
_________________________________________________________________
dense_1 (Dense)              (None, 36)                73764     
Total params: 21,876,548
Trainable params: 73,764
Non-trainable params: 21,802,784
_________________________________________________________________


In [11]:
wandb.init(project='test', entity='scueroinc')
Fine_Tuning_model.fit(x_train_preprocessed, y_train, epochs=15, validation_data=(x_test_preprocessed, y_test), callbacks=[WandbCallback()])

wandb: Currently logged in as: scueroinc (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.25 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Train on 3348 samples, validate on 1440 samples
Epoch 1/15
3348/3348 [==============================] - 50s 15ms/step - loss: 2.7355 - accuracy: 0.3187 - val_loss: 33.2365 - val_accuracy: 0.0299
Epoch 2/15
3348/3348 [==============================] - 45s 14ms/step - loss: 1.5828 - accuracy: 0.6434 - val_loss: 36.7342 - val_accuracy: 0.0333
Epoch 3/15
3348/3348 [==============================] - 45s 14ms/step - loss: 1.1515 - accuracy: 0.7315 - val_loss: 44.4939 - val_accuracy: 0.0347
Epoch 4/15
3348/3348 [==============================] - 45s 14ms/step - loss: 0.9251 - accuracy: 0.7954 - val_loss: 49.0435 - val_accuracy: 0.0361
Epoch 5/15
3348/3348 [==============================] - 45s 14ms/step - loss: 0.7882 - accuracy: 0.8289 - val_loss: 50.3579 - val_accuracy: 0.0285
Epoch 6/15
3348/3348 [==============================] - 45s 14ms/step - loss: 0.6804 - accuracy: 0.8459 - val_loss: 51.9990 - val_accuracy: 0.0326
Epoch 7/15
3348/3348 [==============================] - 45s 14ms/step 

In [12]:
# We can allow some of the resnet layers to change as we train.  
# Typically you would want to lower the learning rate in conjunction with this.

Fine_Tuning_model.layers[0].trainable = True

# We let the last 3 blocks train
for layer in Fine_Tuning_model.layers[0].layers[:-11]:
    layer.trainable = False
for layer in Fine_Tuning_model.layers[0].layers[-11:]:
    layer.trainable = True
    
Fine_Tuning_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
Fine_Tuning_model.fit(x_train_preprocessed, y_train, epochs=15, validation_data=(x_test_preprocessed, y_test), callbacks=[WandbCallback()])

Train on 3348 samples, validate on 1440 samples
Epoch 1/15
3348/3348 [==============================] - 52s 15ms/step - loss: 0.3105 - accuracy: 0.9313 - val_loss: 0.9273 - val_accuracy: 0.7229
Epoch 2/15
3348/3348 [==============================] - 46s 14ms/step - loss: 0.2625 - accuracy: 0.9459 - val_loss: 0.9123 - val_accuracy: 0.7569
Epoch 3/15
3348/3348 [==============================] - 46s 14ms/step - loss: 0.2264 - accuracy: 0.9576 - val_loss: 0.8838 - val_accuracy: 0.7549
Epoch 4/15
3348/3348 [==============================] - 46s 14ms/step - loss: 0.2279 - accuracy: 0.9582 - val_loss: 0.9003 - val_accuracy: 0.7535
Epoch 5/15
3348/3348 [==============================] - 46s 14ms/step - loss: 0.2218 - accuracy: 0.9549 - val_loss: 0.8813 - val_accuracy: 0.7597
Epoch 6/15
3348/3348 [==============================] - 46s 14ms/step - loss: 0.2038 - accuracy: 0.9630 - val_loss: 0.8551 - val_accuracy: 0.7653
Epoch 7/15
3348/3348 [==============================] - 46s 14ms/step - loss

In [13]:
from keras.preprocessing import image
from keras.utils.vis_utils import plot_model
from keras.models import load_model
import numpy as np
import h5py
print("Funciona inicio")
model = keras.models.load_model("best.h5")
print("Funciona load")
img = image.load_img('elephant.jpg', target_size=(299, 299))
print("Funciona carga")
x = image.img_to_array(img)
print("Funciona to array")
x = np.expand_dims(x, axis=0)
print("Funciona dims")
x = preprocess_input(x)
print("Funciona input")
preds = model.predict(x)
print("Funciona predict")
print(preds)

print('Predicted:', decode_predictions(preds, top=3)[0])

Funciona inicio


OSError: Unable to open file (unable to open file: name = 'best.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)